In [2]:
import xlwings

In [3]:
xlwings.sheets

Sheets([<Sheet [통합 문서1]Sheet1>])

In [4]:
xlwings.Range("A1")

<Range [통합 문서1]Sheet1!$A$1>

In [5]:
xlwings.Range("A1").expand()

<Range [통합 문서1]Sheet1!$A$1:$B$10>

In [7]:
total_rows = xlwings.Range("A1").expand().value
total_rows

[['message', 'browser'],
 ['로그1', 'chrome'],
 ['로그2', 'chrome'],
 ['로그3', 'chrome'],
 ['로그4', 'chrome'],
 ['로그5', 'chrome'],
 ['로그6', 'chrome'],
 ['로그7', 'chrome'],
 ['로그8', 'chrome'],
 ['로그9', 'chrome']]

In [8]:
column_names = total_rows[0]
column_names

['message', 'browser']

In [9]:
total_rows[1:]

[['로그1', 'chrome'],
 ['로그2', 'chrome'],
 ['로그3', 'chrome'],
 ['로그4', 'chrome'],
 ['로그5', 'chrome'],
 ['로그6', 'chrome'],
 ['로그7', 'chrome'],
 ['로그8', 'chrome'],
 ['로그9', 'chrome']]

In [10]:
for row in total_rows[1:]:
        row_dict = dict(zip(column_names, row))
        print(row_dict)

{'message': '로그1', 'browser': 'chrome'}
{'message': '로그2', 'browser': 'chrome'}
{'message': '로그3', 'browser': 'chrome'}
{'message': '로그4', 'browser': 'chrome'}
{'message': '로그5', 'browser': 'chrome'}
{'message': '로그6', 'browser': 'chrome'}
{'message': '로그7', 'browser': 'chrome'}
{'message': '로그8', 'browser': 'chrome'}
{'message': '로그9', 'browser': 'chrome'}


In [12]:
# list comprehension
rows_data = [
    dict(zip(column_names, row))
    for row in total_rows[1:]
]
rows_data

[{'message': '로그1', 'browser': 'chrome'},
 {'message': '로그2', 'browser': 'chrome'},
 {'message': '로그3', 'browser': 'chrome'},
 {'message': '로그4', 'browser': 'chrome'},
 {'message': '로그5', 'browser': 'chrome'},
 {'message': '로그6', 'browser': 'chrome'},
 {'message': '로그7', 'browser': 'chrome'},
 {'message': '로그8', 'browser': 'chrome'},
 {'message': '로그9', 'browser': 'chrome'}]

# 정리

In [16]:
def get_rows_data():
    total_rows = xlwings.Range("A1").expand().value
    column_names = total_rows[0]
    rows_data = [
        dict(zip(column_names, row))
        for row in total_rows[1:]
    ]
    return rows_data

get_rows_data()

[{'message': '로그1', 'browser': 'chrome'},
 {'message': '로그2', 'browser': 'chrome'},
 {'message': '로그3', 'browser': 'chrome'},
 {'message': '로그4', 'browser': 'chrome'},
 {'message': '로그5', 'browser': 'chrome'},
 {'message': '로그6', 'browser': 'chrome'},
 {'message': '로그7', 'browser': 'chrome'},
 {'message': '로그8', 'browser': 'chrome'},
 {'message': '로그9', 'browser': 'chrome'}]

# 데이터베이스 접속

In [17]:
DATABASE = {
    "host" : "127.0.0.1",
    "user" : "root",
    "password" : "1234",
    "db" : "myproj11_db",
    "charset" : "utf8",
}

In [22]:
import pymysql

connection = pymysql.connect(**DATABASE)
cursor = connection.cursor()
cursor.execute("SELECT * FROM log;")
print(cursor.fetchall())
cursor.close()

()


# with 절

In [23]:
import pymysql
from contextlib import contextmanager

@contextmanager
def db_cursor():
    connection = pymysql.connect(**DATABASE)
    cursor = connection.cursor()
    try:
        yield cursor
    finally:
        cursor.close()

In [24]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    print(cursor.fetchall())

()


In [33]:
column_data = {'message': '로그1', 'browser': 'chrome'}

with db_cursor() as cursor:
    for row in get_rows_data():
        sql = "INSERT INTO LOG(message, browser) VALUES(%(message)s, %(browser)s)"
        cursor.execute(sql, row_dict)

# SQL 삽입 공격의 예

In [71]:
with db_cursor() as cursor:
    log_id = '1 or 1'
    # sql = "SELECT * FROM log WHERE id = " + str(log_id)
    # sql = "SELECT * FROM log WHERE id = 1 or 1"
    
    sql = "SELECT * FROM log WHERE id = %s"
    cursor.execute(sql, [log_id])
    for row in cursor:
        print(row)

(1, '로그1', 'chrome')


# 데이터 매핑의 예

In [70]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = cursor.fetchall()
#    for row in cursor:
#        row[0] # id
#        row[1] # message
#        row[2] # chrome
#        print(row)
rows_data

((1, '로그1', 'chrome'),
 (2, '로그2', 'chrome'),
 (3, '로그3', 'chrome'),
 (20, '로그4', 'chrome'))

# dict로의 변환

# namedtuple

In [58]:
from collections import namedtuple

Log = namedtuple('Log', 'id message browser')

In [63]:
log = (1, 'message', 'chrome')
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [64]:
log = Log(1, 'message', 'chrome')
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [65]:
log.id, log.message, log.browser

(1, 'message', 'chrome')

In [67]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = [
        Log(*row)
        for row in cursor.fetchall()]
    
rows_data

[Log(id=1, message='로그1', browser='chrome'),
 Log(id=2, message='로그2', browser='chrome'),
 Log(id=3, message='로그3', browser='chrome'),
 Log(id=20, message='로그4', browser='chrome')]

In [25]:
get_rows_data()

[{'message': '로그1', 'browser': 'chrome'},
 {'message': '로그2', 'browser': 'chrome'},
 {'message': '로그3', 'browser': 'chrome'},
 {'message': '로그4', 'browser': 'chrome'},
 {'message': '로그5', 'browser': 'chrome'},
 {'message': '로그6', 'browser': 'chrome'},
 {'message': '로그7', 'browser': 'chrome'},
 {'message': '로그8', 'browser': 'chrome'},
 {'message': '로그9', 'browser': 'chrome'}]

# Python에서 사전 만들 때 참고사항

In [ ]:
d1 = {
    'key1': 'value1',
    'key2': 'value2',
}

d2 = dict(
    key1='value1',
    key2='value2',
)